In [1]:
import spiceypy as spice
import pvl
import os
import re
import subprocess
from ale import util
from itertools import chain
import io
import networkx as nx


# These should be provided when running this script. 
cube = "/work/users/sgstapleton/kernel_split/EN1072174528M.cub"
output_dir = "/work/users/sgstapleton/kernel_split/" # Output dir for created kernel files
data_dir = "/usgs/cpkgs/isis3/data/" # Dir of where to pull original kernels from

    
def get_reference_id(bc_filename):
    brief = subprocess.check_output(["ckbrief", bc_filename])
    reference = None

    for line in brief.splitlines():
        line = str(line)
        if('Object:' in line):
            reference = int(line.strip("''").split()[1])
            
    return reference


def add_light_time_correction(cube_info, kernels):
    
    furnished_kerns = spice.furnsh(kernels)
    
    image_start_et = spice.scs2e(cube_info['SpacecraftID'], cube_info['SpacecraftClockCount'])
    image_end_et = image_start_et + cube_info['ExposureDuration']
    
    inst_state, inst_lt = spice.spkez(cube_info['SpacecraftID'], (image_start_et + image_end_et)/2, 'J2000', 'LT+S', 0)
    target_state, target_lt = spice.spkez(cube_info['TargetID'], (image_start_et + image_end_et)/2, 'J2000', 'LT+S', 0)
    
    lt_pad = max(abs(inst_lt), abs(target_lt)) + 15
    
    # Eph time
    padded_start_et = image_start_et - lt_pad
    padded_end_et = image_end_et + lt_pad
    
    # Padded times
    padded_start_sclk = spice.sce2s(cube_info['SpacecraftID'], padded_start_et)
    padded_end_sclk = spice.sce2s(cube_info['SpacecraftID'], padded_end_et)
    padded_start_utc = spice.et2utc(padded_start_et, 'c', 3)
    padded_end_utc = spice.et2utc(padded_end_et, 'c', 3)

    cube_info.update(PaddedStartTimeSCLK = padded_start_sclk)
    cube_info.update(PaddedEndTimeSCLK = padded_end_sclk)
    cube_info.update(PaddedStartTimeUTC = padded_start_utc)
    cube_info.update(PaddedEndTimeUTC = padded_end_utc)
    

def get_body_graph(spk_file, target_spks=[]):
    brief = subprocess.check_output(["brief", "-c {}".format(spk_file)])
    
    # Convert from bytes
    brief = brief.decode("utf-8")
    brief_io = io.StringIO(brief)
    line = brief_io.readline()
    body_graph = nx.DiGraph()
        
    while(line):
        if("w.r.t" in line):
            bodies_results = re.findall('\((.*?)\)', line)
            body_graph.add_edge(*bodies_results)
                        
        line = brief_io.readline()
    
    if 0 not in body_graph.nodes():
        for spk in target_spks:
            new_body_graph = get_body_graph(spk)
            body_graph.add_edges_from(new_body_graph.edges())
            

    return body_graph


def output_spkmerge_config(config_file, cube_info, output_dir):
    
    with open(config_file, "w+") as config:
        config.write('''LEAPSECONDS_KERNEL     = {}\n'''.format(cube_info['LeapSecond'][0]))
    
        for kern in cube_info['InstrumentPosition']:
            
            basename = os.path.basename(kern).split('.')[0]
            filename, file_extension = os.path.splitext(kern)
            new_kern = output_dir + basename + "_merged" + file_extension
            
            config.write('''      SPK_KERNEL             = {}\n'''.format(new_kern))

            body_graph = get_body_graph(kern, cube_info['TargetPosition'])
            
            body_ids = []
            for path in nx.all_simple_paths(body_graph,  str(cube_info['SpacecraftID']), '0'):
                body_ids = body_ids + path

            body_ids = set(body_ids)

            for body in body_ids:

                if body != '0':

                    config.write('''         SOURCE_SPK_KERNEL   = {}\n'''.format(kern))
                    config.write('''            INCLUDE_COMMENTS = no\n''')
                    config.write('''            BODIES              = {}\n'''.format(body))
                    config.write('''            BEGIN_TIME          = {}\n'''.format(cube_info['PaddedStartTimeUTC']))
                    config.write('''            END_TIME            = {}\n'''.format(cube_info['PaddedEndTimeUTC']))
                    
    config.close()
    

In [2]:
# These are the processing steps. This will make use of the cube provided further up to create smaller, 
# more manageable kernel files for ale testing purposes. This currently only handles ck and spk files.

# Get dictionary of kernel lists from cube
cube_info = util.generate_kernels_from_cube(cube, format_as='dict')

# Replace path variables with absolute paths for kernels
for kernel_list in cube_info:
    for index, kern in enumerate(cube_info[kernel_list]):
        if kern is not None:
            cube_info[kernel_list][index] = data_dir + kern.strip('$')
            
# Create ordered list of kernels for furnishing
kernels = [kernel for kernel in chain.from_iterable(cube_info.values()) if isinstance(kernel, str)]
                
# Loads cube as pvl to extract rest of data
cube_pvl = pvl.load(cube)
               
# Save other necesary info in cube_info dict
cube_info.update(SpacecraftClockCount = cube_pvl['IsisCube']['Instrument']['SpacecraftClockCount'])

try:
    cube_info.update(ExposureDuration = cube_pvl['IsisCube']['Instrument']['ExposureDuration'].value * 0.001)
except:
    cube_info.update(ExposureDuration = cube_pvl['IsisCube']['Instrument']['LineExposureDuration'].value * 0.001)
    
cube_info.update(TargetID = spice.bods2c(cube_pvl['IsisCube']['Instrument']['TargetName']))

spacecraft_lookup = {'Mars_Reconnaissance_Orbiter' : 'MRO'}
try:
    cube_info.update(SpacecraftID = spice.bods2c(cube_pvl['IsisCube']['Instrument']['SpacecraftName']))
except:
    spacecraft_name = cube_pvl['IsisCube']['Instrument']['SpacecraftName']
    cube_info.update(SpacecraftID = spice.bods2c(spacecraft_lookup[spacecraft_name]))

# Add lighttime-corrected SCLK values to cube_info
add_light_time_correction(cube_info, kernels)

# For each binary ck kernel specified in cube, run the ckslicer, comment and to-transfer commands
for ck in cube_info['InstrumentPointing']:
    if ck.endswith('.bc'):
        
        # Get reference id associated with ck
        reference_id = get_reference_id(ck)
        
        ck_filename = os.path.basename(ck).split('.')[0]
        ck_path, ck_file_extension = os.path.splitext(ck)
        output_kern = output_dir + ck_filename + '_sliced' + ck_file_extension
        
        # Create new sliced ck kernel
        ckslicer_command = ["./ckslicer", 
                                '-LSK {}'.format(cube_info['LeapSecond'][0]), 
                                '-SCLK {}'.format(cube_info['SpacecraftClock'][0]), 
                                '-INPUTCK {}'.format(ck), 
                                '-OUTPUTCK {}'.format(output_kern),
                                '-ID {}'.format(reference_id),
                                '-TIMETYPE {}'.format('SCLK'),
                                '-START {}'.format(cube_info['PaddedStartTimeSCLK']),
                                '-STOP {}'.format(cube_info['PaddedEndTimeSCLK'])]
        subprocess.call(ckslicer_command)
        
        # Remove old comments from new ck kernel
        commnt_command = ['commnt', '-d {}'.format(output_kern)]
        subprocess.call(commnt_command)
        
        # Makes a new txt file for the only comments that should be stored in the new ck file
        with open("temp_commnts.txt","w+") as f:
            f.write("This CK is for testing with the image: {}\n".format(cube))
            f.write("\nThis CK was generated using the following command: {}\n".format(" ".join(ckslicer_command)))
        
        # Add new comments to new ck kernel
        new_commnts_command = ["commnt", "-a {}".format(output_kern), "temp_commnts.txt"]
        subprocess.call(new_commnts_command)
        
        # Create the transfer file of the new ck kernel
        subprocess.call(["toxfr", output_kern])

# Create the config file for the spkmerge command
output_spkmerge_config(output_dir + "spk.config", cube_info, output_dir)

# Run the spkmerge command
spkmerge_command = ["spkmerge", output_dir + "spk.config"]
subprocess.call(spkmerge_command)

# Clean up temp config file
# os.remove(output_dir + "spk.config")

In [4]:
{
     'radii': {'semimajor': 3396.19, 'semiminor': 3376.2, 'unit': 'km'},
     'sensor_position': {'positions': [[-615012.0886971647, -97968.2345594813, -3573947.032011338],
                                       [-615520.6109230528, -97906.4784443392, -3573862.281296898],
                                       [-616029.119550515, -97844.70954517406, -3573777.458632478],
                                       [-616537.6144124779, -97782.9278549998, -3573692.564075001],
                                       [-617046.0958326485, -97721.13339810426, -3573607.5975138554],
                                       [-617554.5636389386, -97659.32615734483, -3573522.559012526]],
                         'velocities': [[-3386.5803072963226, 411.22659677345894, 564.1630407463263],
                                       [-3386.4898408011636, 411.3117338896111, 564.6421991495939],
                                       [-3386.3993050254394, 411.39686089826347, 565.1213459594977],
                                       [-3386.3087000099817, 411.4819777303558, 565.600480994323],
                                       [-3386.218025688375, 411.56708444186563, 566.0796046032438],
                                       [-3386.127282099038, 411.65218101277696, 566.5587166016444]],
                         'unit': 'm'},
     'sun_position': {'positions': [[-127052102329.16032, 139728839049.65073, -88111530293.94502]],
                      'velocities': [[9883868.06162645, 8989183.29614645, 881.9339912834714]],
                      'unit': 'm'},
     'sensor_orientation': {'quaternions': [[0.0839325155418465, 0.01773153459973076, 0.9946048838768001, 0.05832709905329919],
                                            [0.08400255389846112, 0.017728573660888338, 0.9945982552324419, 0.058340203145592955],
                                            [0.0840727438677933, 0.01772597983213093, 0.9945916990701015, 0.058351653947255284],
                                            [0.08414295024563542, 0.0177232314764712, 0.9945850926215932, 0.058363897444302155],
                                            [0.08421298603777543, 0.017720474751607065, 0.9945784602692478, 0.05837674302007016],
                                            [0.08428244063999622, 0.01771827039990229, 0.9945719105583338, 0.058388764519787986]]},
     'detector_sample_summing': 1,
     'detector_line_summing': 1,
     'focal_length_model': {'focal_length': 352.9271664},
     'detector_center': {'line': 0.430442527, 'sample': 2542.96099},
     'starting_detector_line': 0,
     'starting_detector_sample': 0,
     'focal2pixel_lines': [0.0, 142.85714285714, 0.0],
     'focal2pixel_samples': [0.0, 0.0, 142.85714285714],
     'optical_distortion': {'radial': {'coefficients': [-0.0073433925920054505, 2.8375878636241697e-05, 1.2841989124027099e-08]}},
     'image_lines': 400,
     'image_samples': 5056,
     'name_platform': 'MARS_RECONNAISSANCE_ORBITER',
     'name_sensor': 'CONTEXT CAMERA',
     'reference_height': {'maxheight': 1000, 'minheight': -1000, 'unit': 'm'},
     'name_model': 'USGS_ASTRO_LINE_SCANNER_SENSOR_MODEL',
     'interpolation_method': 'lagrange',
     'line_scan_rate': [[0.5, -0.37540000677108765, 0.001877]],
     'starting_ephemeris_time': 297088762.24158406,
     'center_ephemeris_time': 297088762.61698407,
     't0_ephemeris': -0.37540000677108765,
     'dt_ephemeris': 0.15016000270843505,
     't0_quaternion': -0.37540000677108765,
     'dt_quaternion': 0.15016000270843505}

{'radii': {'semimajor': 3396.19, 'semiminor': 3376.2, 'unit': 'km'},
 'sensor_position': {'positions': [[-615012.0886971647,
    -97968.2345594813,
    -3573947.032011338],
   [-615520.6109230528, -97906.4784443392, -3573862.281296898],
   [-616029.119550515, -97844.70954517406, -3573777.458632478],
   [-616537.6144124779, -97782.9278549998, -3573692.564075001],
   [-617046.0958326485, -97721.13339810426, -3573607.5975138554],
   [-617554.5636389386, -97659.32615734483, -3573522.559012526]],
  'velocities': [[-3386.5803072963226, 411.22659677345894, 564.1630407463263],
   [-3386.4898408011636, 411.3117338896111, 564.6421991495939],
   [-3386.3993050254394, 411.39686089826347, 565.1213459594977],
   [-3386.3087000099817, 411.4819777303558, 565.600480994323],
   [-3386.218025688375, 411.56708444186563, 566.0796046032438],
   [-3386.127282099038, 411.65218101277696, 566.5587166016444]],
  'unit': 'm'},
 'sun_position': {'positions': [[-127052102329.16032,
    139728839049.65073,
    -8811